In [7]:
import numpy as np
import math

In [8]:
def d2blmax_2pwr(d,p1,p2,c2b,dbh_maxh):
    '''input dbh [cm]
    output: plant leaf biomass [kg C]'''
    blmax = []
    for i in d:
        blmax.append((p1*min(i,dbh_maxh)**p2) / c2b)   
    return blmax

def d_to_crown_area_fates(ds,d2bl_p2,d2bl_ediff,spread,d2ca_max,d2ca_min, dbh_maxh):
    
    '''Input dbh (cm)
    Output: crown area (m2)'''
    
    ca = []
    cd = []
    
    for d in ds:
        #from fates
        
        if d > dbh_maxh:
            d = dbh_maxh
        
        crown_area_to_dbh_exponent = d2bl_p2 + d2bl_ediff
        spreadterm = spread * d2ca_max + (1.0 - spread) * d2ca_min
        tmp_ca = spreadterm * d ** crown_area_to_dbh_exponent
        ca.append(tmp_ca)

        #convert to crown diameter
        cd.append(2 * math.sqrt(tmp_ca/math.pi))

    return ca,cd

In [21]:
def vai_check(diams,d2bl1,d2bl2,d2ca,ediff,sai_per_lai,dbh_maxh,slamax,c2b,vai_max=30,print_leaf_area = False):
    '''
    Returns the total vai of a tree and a warning of the vai is greater than the allowed maximum.
    
    
    Params
    diams = range of dbh to check vai for,
    d2bl1 = dbh to leaf biomass p1
    d2bl2 = dbh to leaf biomass p2
    d2ca = diameter to crown area parameter (base of the d2ca 2pwr function)
    ediff = exponent different for the d2ca function
    sai_per_lai = how much stem area index per lai
    dbh_maxh = dbh at maximum height
    slamax = maximum specific leaf area (m2 per g of carbon)
    c2b = grams of biomass per g of carbon
    vai_max = the allowed maximum vai for a tree
    '''
    
    print("Using the following parameters:\n")
    print("d2bl1",d2bl1)
    print("d2bl2",d2bl2)
    print("d2ca",d2ca)
    print("ediff",ediff)
    print("sai_per_lai",sai_per_lai)
    print("dbh_maxh",dbh_maxh)
    print("slamax",slamax)
    print("c2b",c2b)
    
    
    # Calculate total tree leaf area (m2)
    leaf_biomass = d2blmax_2pwr(d = diams,
                            p1 = d2bl1,
                            p2 = d2bl2,
                            c2b=c2b,
                            dbh_maxh = dbh_maxh)
    leaf_biomass_g = np.array(leaf_biomass) * 1000
    leaf_area_m2 = leaf_biomass_g * slamax
    
    if print_leaf_area == True:
        print("leaf_area_m2",leaf_area_m2)
    
    
    # Calculate tree projected crown area (ca) and diameter (cd)
    ca,cd = d_to_crown_area_fates(ds=diams,
                              d2bl_p2=d2bl2,
                              d2bl_ediff=ediff,
                              d2ca_max=d2ca,
                              d2ca_min=d2ca,
                              spread=0.5,
                              dbh_maxh=dbh_maxh)

    print("ca",ca)
    tree_vai = (np.array(leaf_area_m2) / np.array(ca)) * (1 + sai_per_lai)
    print("vai",tree_vai)
    
    max_tree_vai = np.max(tree_vai)
    
    print("\nVAI check:")
    
    if max_tree_vai > vai_max:
        print("Warning: tree vai of",max_tree_vai,"exceeds vai max of",vai_max)
    else:
        print("Max vai",max_tree_vai,"is OK")

In [33]:
vai_check(diams = [90,388.807],
          d2bl1 = 0.020336868,
          d2bl2 = 2.01121055798523,
          d2ca = 0.284979607940355,
          ediff = -0.793196,
          sai_per_lai = 0.1,
          dbh_maxh = 388.80,
          slamax = 0.0165065166847673,
          c2b = 2,
          vai_max=50,
          print_leaf_area = True)

Using the following parameters:

d2bl1 0.020336868
d2bl2 2.01121055798523
d2ca 0.284979607940355
ediff -0.793196
sai_per_lai 0.1
dbh_maxh 388.8
slamax 0.0165065166847673
c2b 2
leaf_area_m2 [ 1429.89015889 27126.53346144]
ca [68.40842805834286, 406.57256155300405]
vai [22.9924765  73.39203288]

VAI check:


In [32]:
1/.4

2.5

In [18]:
27126.53346144/406.57256155300405

66.72002989533658